In [224]:
import pickle
import pandas as pd 
import bokeh.palettes as palettes
import sys
sys.path.insert(0, '../scripts/')
import coref_resolution_util as c
from more_itertools import unique_everseen
from copy import deepcopy
from IPython.display import display, HTML
import numpy as np
from importlib import reload
reload(c)

<module 'coref_resolution_util' from '../scripts/coref_resolution_util.py'>

In [225]:
fname = '../data/output/newssniffer-nytimes/output_chunk__start-15_end-2015.pkl'
with open(fname, 'rb') as f:
    processed = pickle.load(f)

In [227]:
processed_df = (
    pd.DataFrame(processed)
    .drop_duplicates('entry_id')
)

In [137]:
idx = 0
q = processed_df.iloc[idx]['quote_idxes']
s = processed_df.iloc[idx]['sent_parse']
l = list(map(len, s))

In [176]:
if len(q) < 10:
    cmap = palettes.Set3_11
elif len(q) < 20:
    cmap = list(unique_everseen(palettes.Set3_11 + palettes.Spectral11))
else:
    cmap = list(unique_everseen( palettes.Set3_11 + palettes.Spectral11 + palettes.Category20_20))

comp_mapper = {i:idx for idx, i in enumerate(q.index.tolist())}
source_idx_head_mapper = (
 q
 .assign(head=lambda df: df.apply(lambda x: ' '.join(s[x['head_s_idx']][
        x['head_span'][0] - sum(l[:x['head_s_idx']]):
        x['head_span'][1] - sum(l[:x['head_s_idx']])
    ]
    ), axis=1 ))
    ['head'].to_dict()
)

span_df = (
 q
 .assign(q=lambda df: df.apply(lambda x: 
   list(map(lambda y: {'source': x.name, 'sent': y[0], 'span': y[1], 'type': 'quote'}, x['quote_chunks'] )), axis=1))
 .assign(b=lambda df: df.apply(lambda x: 
   list(map(lambda y: {'source': x.name, 'sent': y[0], 'span': y[1], 'type': 'background'}, x['background_chunks'] )), axis=1))
 .apply(lambda x: x['q'] + x['b'], axis=1)
 .pipe(lambda s: pd.DataFrame(c.flatten_list_of_lists(s.tolist())))
 .sort_values('sent', ascending=False)
 .assign(head=lambda df: df['source'].map(source_idx_head_mapper))
 .assign(source=lambda df: df['source'].map(comp_mapper))
)

In [202]:
s_html = deepcopy(s)
old_s_idx = None
# quotes and backgrounds
for idx, (source_idx, sent_idx, quote_span, quote_type, head) in span_df.iterrows():
    if old_s_idx != sent_idx:
        adj = sum(l[:sent_idx])
    sent, adj = c.format_span_with_word_list(adj=adj, sent=s[sent_idx], span=quote_span, bold=True)
    s_html[sent_idx] = {
        'sent':' '.join(sent),
        'type': quote_type.upper(),
        'head': head,
        'color': cmap[source_idx],
        'source_idx': source_idx
    }

# regular
for s_idx, sent in enumerate(s_html):
    if isinstance(sent, list):
        s_html[s_idx] = {
            'sent': ' '.join(sent), 
            'type': '',
            'source_idx': 'null'
        }

In [183]:
display(HTML('<br><br>'.join(s_html).replace('$', '\$')))

TypeError: sequence item 0: expected str instance, dict found

In [203]:
import json
with open('../app/data/test-json.json', 'w') as f:
    json.dump(s_html, f)

# Examine output data

In [204]:
with open('../app/data/output-json.json') as f:
    t = json.load(f)

In [209]:
t.pop('start_time', '')

'1645483752.553241'

In [211]:
sorted_keys = sorted(t.keys(), key=lambda x: int(x))

In [214]:
t_list = list(map(lambda k: t[k], sorted_keys))

In [219]:
t_df = pd.DataFrame(t_list)
t_df.head()

,s_idx,sentence,head,quote_type,error,affiliation,role,gender,race,age,education
0,0,"SANTIAGO , Chile —",,,No Errors,,,,,,
1,1,They appear at the student demonstrations that...,,,No Errors,,,,,,
2,2,They are not there to join the protests or int...,,,No Errors,,,,,,
3,3,"This month , they are busier than ever .",,,No Errors,,,,,,
4,4,"The volunteer observers , known as “ helmets ,...",,,No Errors,,,,,,


In [223]:
(t_df
 .groupby('head')
 [['affiliation', 'role', 'gender', 'race', 'age', 'education']]
 .aggregate(list)
 .applymap(lambda x: list(set(x)))
)

,affiliation,role,gender,race,age,education
head,,,,,,
,[],[],[],[],[],[]
Germán Chau,[NGO],[Decision Maker],[],[],[66],[]
Luis Parra,"[, NGO]","[, Decision Maker]","[, Male]","[, Latinx]","[, 56]","[Law, ]"
Marta Cisterna,"[, NGO]","[, Informational]","[, Female]","[, Latinx]","[, 45]",[]
Matías Sotelo,"[, NGO]","[, Decision Maker]","[, Male]","[, Latinx]","[, 23]","[, MD]"
